In [21]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [22]:
!pip install transformers sentencepiece torch datasets sentence-transformers tokenizers simplet5 evaluate #modelcard#ml_things
!pip install jiwer 
!pip install tensorflow
!pip install inseq

!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.3
    Uninstalling transformers-4.27.3:
      Successfully uninstalled transformers-4.27.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inseq 0.4.0 requires transformers[sentencepiece,tokenizers,torch]<5.0.0,>=4.22.0, but you have transformers 4.16.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: proto

In [ ]:
# if you have gpu which is cuda compatible run this
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Dataset creation

In [ ]:
from datasets import load_dataset, load_from_disk
import os 

MODEL_CHECKPOINT = 'google/byt5-base'

def load_data(file_path, load_from_local = True):
    """ Load data from .tsv or previously saved tokenized dataset
    :param file_path: str, absolute location of the path where data is located
    :param load_from_local: boolean, true if want to load saved dataset
    """
    if not load_from_local: 
        print("#### Creating and tokenizing dataset #### ")
        print
        dataset = load_dataset("csv", data_files={  
                                                    "train": file_path + "\\deu_600.csv",
                                                    "test": file_path + "\\deu_gold.csv", 
                                                    "validation": file_path + "\\deu_dev.csv"    
                                                }
                                )

        print(f"#### Removed {dataset.cleanup_cache_files()} cached files")    
        
        # uncomment below 3 lines for local testing
        # dataset["train"] = dataset["train"].select(range(0,4))
        # dataset["test"] = dataset["test"].select(range(0,4))
        # dataset["validation"] = dataset["validation"].select(range(0,4))
        
        dataset = preprocess(dataset)
        dataset.save_to_disk(file_path + "\\german_noun_pl.hf")
    
    else:
        print("#### Loading dataset from local #### ")
        dataset = load_from_disk(file_path + "\\german_noun_pl.hf")
        # uncomment below 3 lines for local testing
        # dataset["train"] = dataset["train"].select(range(0,3))
        # dataset["test"] = dataset["test"].select(range(0,3))
        # dataset["validation"] = dataset["validation"].select(range(0,3))

    return dataset

def format_input(example):
    """ format the lemma and tag columns into {lemma, tag}
    tags are in UniMorph format
    """
    return {"inflection": {"input": example["lemma"], "tag": example["tag"],
            "output": example["target_form"]}}

# https://huggingface.co/docs/datasets/use_dataset
def preprocess(dataset):
    for split in dataset.keys():
        dataset[split] = dataset[split].map(format_input, load_from_cache_file=False)
        dataset[split] = dataset[split].remove_columns(["lemma", "tag", "target_form"])
        
    return dataset

In [ ]:
# file_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/final_project/german_data" # might need to change depending on where your datasets + code is
filepath = os.getcwd()
print(filepath)
dataset = load_data(filepath + "\\german_data", True) # make false if you want to tokenize data again

# dataset = load_data(filepath + "\\german_data", True) # False if you want to reload the dataset

c:\Users\jeroe\Documents\NLP-project-main
#### Loading dataset from local #### 


In [ ]:
# dataset not tokenised yet
print(dataset)
print(dataset["train"])
print(dataset["train"]["inflection"][0]) # first elem 

DatasetDict({
    train: Dataset({
        features: ['inflection'],
        num_rows: 600
    })
    test: Dataset({
        features: ['inflection'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['inflection'],
        num_rows: 500
    })
})
Dataset({
    features: ['inflection'],
    num_rows: 600
})
{'input': 'Plätzchen', 'output': 'Plätzchen', 'tag': 'N;NOM;NEUT;PL'}


# Model finetuning

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
import evaluate
import gc
import numpy as np

MODEL_CHECKPOINT = "google/byt5-base"

def preprocess_function(examples):
    """ To process the input before using it in model (tokenization)
    """
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    max_input_length = 128
    max_target_length = 128

    inputs = [ex["tag"] + ": " + ex["input"] for ex in examples["inflection"]]
    targets = [ex["output"] for ex in examples["inflection"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def train_transformer(dataset, model_name, model_dir, num_epochs = 3):
    """ Finetune the byT5 transformer model and save it at model_dir
    """

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    print(f"#### training on {device} ####")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    print("#### Tokenizing dataset ####")
    tokenized_datasets = dataset.map(preprocess_function, batched=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
    model.to(device)

    n_beams = 3
    max_gen_len = 36 # longest german noun has 36 letters
    num_epochs = 3
    train_batch_size = 2 # change it to something lower if you get memory error
    eval_batch_size =2 


    args = Seq2SeqTrainingArguments(
        output_dir=model_dir + "\\" + model_name + "\\checkpoints",
        evaluation_strategy = "epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=eval_batch_size,
        weight_decay=0.01,
        save_total_limit = 3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        metric_for_best_model="cer", # metric to reduce (Character Error Rate)
        generation_max_length=max_gen_len,
        optim="adafactor",
        generation_num_beams=n_beams, # for beam search in decoder
        logging_strategy = "epoch",
        logging_dir = model_dir + "\\" + model_name + "\\logging"
    )

    # args = Seq2SeqTrainingArguments(
    #     output_dir=model_dir + "\\" + model_name + "\\checkpoints",
    #     evaluation_strategy = "steps",
    #     eval_steps = 100,
    #     save_strategy='steps',
    #     learning_rate=5e-5,
    #     per_device_train_batch_size=train_batch_size,
    #     per_device_eval_batch_size=eval_batch_size,
    #     weight_decay=0.01,
    #     num_train_epochs=num_epochs,
    #     predict_with_generate=True,
    #     generation_num_beams=n_beams, # for beam search in decoder
    #     metric_for_best_model="cer", # metric to reduce (Character Error Rate)
    #     generation_max_length=max_gen_len,
    #     warmup_steps=500,                                                              
    #     load_best_model_at_end = True,
    #     logging_dir = model_dir + "\\" + model_name + "\\logging",
    #     logging_strategy = "steps",
    #     logging_steps = 100,
    #     save_steps = 100
    # )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    metric = evaluate.load("cer") # metric used by byt5 (character error rate)

    ########
    def postprocess_text(preds, labels):
        """ Format the output so that metrics can be computed
        """
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        return preds, labels

    def compute_metrics(eval_preds):
        """ Compute metrics based on CER
        """
        preds, labels = eval_preds
        
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        print(decoded_preds, decoded_labels)

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"cer": result}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    ########

    trainer = Seq2SeqTrainer(
                                model,
                                args,
                                train_dataset=tokenized_datasets["train"],
                                eval_dataset=tokenized_datasets["validation"],
                                data_collator=data_collator,
                                tokenizer=tokenizer,
                                compute_metrics=compute_metrics
                            )

    print("#### Training model ####")
    train_result = trainer.train()
    trainer.save_model(output_dir=model_dir + "\\" + model_name)

def eval_transformer(dataset, file_path, model_name):
    """ TODO: can go further with CER: find CER for the number of characters in input
    probably its the case that as the no. of charecters increases, the CER increases 
    """
    model = AutoModelForSeq2SeqLM.from_pretrained(file_path + "\\" + model_name)
    # device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # model.to(device)
    # print(f"#### evaluating on {device} ####")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    print("#### Tokenizing dataset ####")
    dataset = dataset["test"]["inflection"]
    input_model = []
    labels = []
    for i in range(len(dataset)):
        input_model.append(dataset[i]["tag"] + ": " + dataset[i]["input"])
        labels.append(dataset[i]["output"])

    model_inputs = tokenizer(input_model, padding="max_length", max_length=36, return_tensors="pt")
    input_ids = model_inputs.input_ids
    attention_mask = model_inputs.attention_mask
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=36)


    decoded_outputs = tokenizer.batch_decode(outputs.tolist(), skip_special_tokens=True)
    print("#### Calculating accuracy and cer score ####")
    # TODO: check if spaces since that would affect direct accuracy 

    # first get accuracy
    # metric = evaluate.load("bleu")
    metric = evaluate.load("chrf") # bleu score usually used for translation like tasks
                                   # idk which one we need to use so i used bleu
                                   # chrf might also be useful
    accuracy_score = metric.compute(predictions=decoded_outputs, references=labels)
    print("Accuracy on test set: ", accuracy_score)
    # Accuracy on test set:  {'bleu': 0.0, 'precisions': [0.5683333333333334, 0.0, 
    # 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 
    # 600, 'reference_length': 600}
    # (chrf) Accuracy on test set:  {'score': 84.37330175800278, 'char_order': 6, 'word_order': 0, 'beta': 2}

    direct_acc = 0
    for i in range(len(decoded_outputs)):
        if decoded_outputs[i] == labels[i]: # not lower cause german nouns are always capital
            direct_acc += 1
    print("Direct accuracy (%) on test set: ", ((direct_acc/len(decoded_outputs)) * 100))
    # direct accuracy is around 56.83% when training on small byt5 and german data 

    # then get CER
    metric = evaluate.load("cer")
    cer_score = metric.compute(predictions=decoded_outputs, references=labels)    
    print("CER on test set: ", cer_score) # lower is better
    # CER on test set:  0.07972233908287758

In [ ]:
gc.collect() # try out to remove cache

12563

In [ ]:
torch.cuda.empty_cache() # try out to remove cache

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   7197 MiB |   7201 MiB |   9982 MiB |   2784 MiB |\n|       from large pool |   7196 MiB |   7200 MiB |   9217 MiB |   2021 MiB |\n|       from small pool |      0 MiB |    195 MiB |    764 MiB |    763 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   7197 MiB |   7201 MiB |   9982 MiB |   2784 MiB |\n|       from large pool |   7196 MiB |   7200 MiB |

In [ ]:
name = "byt5finetune_ruhi"
file_path = os.getcwd() + "\\models"
epochs = 3
train_transformer(dataset, name, file_path, epochs)
# if training on cpu is shown change hardware accelerator to GPU in Runtime > Change runtime type > Hardware Accelerator 

#### training on cuda ####


loading configuration file config.json from cache at C:\Users\jeroe/.cache\huggingface\hub\models--google--byt5-base\snapshots\92d8c008d55cf7c254915bac165171dfe6c20c44\config.json
Model config T5Config {
  "_name_or_path": "google/byt5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3968,
  "d_kv": 64,
  "d_model": 1536,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 12,
  "num_layers": 18,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "transformers_version": "4.26.0",
  "use_cache": true,
  "vocab_size": 3

#### Tokenizing dataset ####


loading configuration file config.json from cache at C:\Users\jeroe/.cache\huggingface\hub\models--google--byt5-base\snapshots\92d8c008d55cf7c254915bac165171dfe6c20c44\config.json
Model config T5Config {
  "_name_or_path": "google/byt5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3968,
  "d_kv": 64,
  "d_model": 1536,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 12,
  "num_layers": 18,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "transformers_version": "4.26.0",
  "use_cache": true,
  "vocab_size": 3

#### Training model ####


...

{'loss': 0.2054, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/250 [00:00<?, ?it/s]

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_

['Flitzer', 'Broschen', 'Chinesen', 'Kloster', 'Urläube', 'Buchstäben', 'Detektive', 'Adler', 'Zweige', 'Schoße', 'Marmeladen', 'Algen', 'Defekte', 'Verbindungen', 'Kompässe', 'Bewegungen', 'Partien', 'Pilze', 'Echsen', 'Schenkel', 'Brombeeren', 'Objekte', 'Bürger', 'Kartone', 'Clowne', 'Depote', 'Fahnen', 'Gehäusen', 'Fausten', 'Opae', 'Westen', 'Aufwände', 'Diäten', 'Frosche', 'Amte', 'Rüstungen', 'Flunder', 'Hyänen', 'Seelen', 'Ausrufezeichen', 'Klingel', 'Fischer', 'Dochte', 'Ruberblicke', 'Kelche', 'Abende', 'Bühnen', 'Beweise', 'Wampen', 'Aste', 'Frisuren', 'Schwäben', 'Ströhhälme', 'Fieber', 'Krankheiten', 'Anführer', 'Klippen', 'Schniedel', 'Biber', 'Matche', 'Schweinerei', 'Verläge', 'Planetariume', 'Stäte', 'Tukane', 'Außerungen', 'Büchereien', 'Detäile', 'Spritzfische', 'Waren', 'Gedärme', 'Abschlüsse', 'Spieler', 'Versprechen', 'Kajüten', 'Präsidente', 'Rückwege', 'Gebüsche', 'Groschen', 'Abgänge', 'Kommoden', 'Nachschube', 'Spiele', 'Aufzüge', 'Gräbe', 'Brüller', 'Achsen',

Model weights saved in c:\Users\jeroe\Documents\NLP-project-main\models\byt5finetune_ruhi\pytorch_model.bin
tokenizer config file saved in c:\Users\jeroe\Documents\NLP-project-main\models\byt5finetune_ruhi\tokenizer_config.json
Special tokens file saved in c:\Users\jeroe\Documents\NLP-project-main\models\byt5finetune_ruhi\special_tokens_map.json


In [ ]:
eval_transformer(dataset, file_path, name)

loading configuration file c:\Users\jeroe\Documents\NLP-project-main\models\byt5finetune_ruhi\config.json
Model config T5Config {
  "_name_or_path": "c:\\Users\\jeroe\\Documents\\NLP-project-main\\models\\byt5finetune_ruhi",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3968,
  "d_kv": 64,
  "d_model": 1536,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 12,
  "num_layers": 18,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "use_cache": true,
  "voca

#### Tokenizing dataset ####


KeyboardInterrupt: 

# Inseq analysis (TODO)

In [25]:
import inseq

def do_contrast_analysis(model_filepath, input_text, contrast_text, truth_text):
    """ Do contrastive analysis as shown in W5E notebook  
    """
    model = inseq.load_model(model_filepath, "input_x_gradient")

    # incorrect one 
    contrast = model.encode(contrast_text, as_targets=True)

    out = model.attribute(
        input_text, # input to model
        truth_text, # Fix the original target
        attributed_fn="contrast_prob_diff",
        # Also show the probability delta between the two options
        step_scores=["contrast_prob_diff", "probability"],
        contrast_ids=contrast.input_ids,
        contrast_attention_mask=contrast.attention_mask,
    )

    # Normally attributions are normalized to sum up to 1
    # Here we want to see how they contribute to the probability difference instead
    out.weight_attributions("contrast_prob_diff")
    out.show()

def do_analysis(model_filepath, input_text):
    """ Do analysis of the prediction (unconstrained) as in W5E notebook
    """
    # not very good probably because of lack of training data?
    # possibly could be because we're not tokenizing the tags and lemma correctly but don't know yet
    model = inseq.load_model(model_filepath, "input_x_gradient")
    out = model.attribute(
        input_text,
        attribute_target=True,
        step_scores=["probability"]
    )
    out.show()

In [23]:
file_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/final_project" # might need to change depending on where your datasets + code is
name = "byt5finetune_ruhi"
input = "N;NOM;NEUT;PL: Kissen"
contrast = "Kissene"
truth = "Kissen"

In [26]:
do_contrast_analysis(file_path + "/models/" + name, input, contrast, truth)

Attributing with input_x_gradient...: 100%|██████████| 8/8 [00:03<00:00,  2.05it/s]


,K,i,s,s,e,n,</s>
N,0.0,0.0,0.0,0.0,0.0,0.0,0.047
;,0.0,0.0,0.0,0.0,0.0,0.0,0.05
N,0.0,0.0,0.0,0.0,0.0,0.0,0.021
O,0.0,0.0,0.0,0.0,0.0,0.0,0.029
M,0.0,0.0,0.0,0.0,0.0,0.0,0.034
;,0.0,0.0,0.0,0.0,0.0,0.0,0.043
N,0.0,0.0,0.0,0.0,0.0,0.0,0.02
E,0.0,0.0,0.0,0.0,0.0,0.0,0.032
U,0.0,0.0,0.0,0.0,0.0,0.0,0.037
T,0.0,0.0,0.0,0.0,0.0,0.0,0.046


In [27]:
do_analysis(file_path + "/models/" + name, input)

Attributing with input_x_gradient...: 100%|██████████| 8/8 [00:01<00:00,  6.33it/s]


,K,i,s,s,e,n,</s>
N,0.032,0.027,0.029,0.028,0.023,0.02,0.027
;,0.056,0.045,0.046,0.038,0.028,0.037,0.029
N,0.03,0.026,0.029,0.021,0.016,0.018,0.012
O,0.033,0.031,0.027,0.023,0.013,0.02,0.017
M,0.051,0.028,0.029,0.028,0.023,0.027,0.02
;,0.06,0.043,0.038,0.032,0.025,0.028,0.025
N,0.03,0.024,0.022,0.021,0.012,0.017,0.011
E,0.025,0.026,0.024,0.023,0.014,0.019,0.019
U,0.032,0.04,0.035,0.034,0.018,0.03,0.022
T,0.027,0.038,0.025,0.039,0.022,0.035,0.027


# Challenge (TODO)